## MSOffice
A comunidade também criou bibliotecas para poder manipular arquivos de MSOffice. Apesar de não serem oficiais (by Microsoft), muitas delas têm muito tempo e funcionam muito bem. Inclusive existe um [site](http://www.python-excel.org/) que ajuda as pessoas a encontrarem as bibliotecas.

In [ ]:
! pip install openpyxl

In [ ]:
import openpyxl
import os
diretorio = 'Office'
arquivo = 'exemplo.xlsx'
wb = openpyxl.load_workbook(os.path.join(diretorio, arquivo))
wb.sheetnames

In [ ]:
# pega uma planilha dentro do arquivo (sheet)
sheet = wb['Plan3']

In [ ]:
# pegando o valor de uma célula somente
print("Valor célula B10: {}".format(sheet['B10'].value))

# pegando a coluna
sheet['C'][1:5]

In [ ]:
# pegando a linha
sheet[1]

In [ ]:
# mostra as 10 primeiras linhas
count = 0
for row in sheet.iter_rows():
    print([ cell.value for cell in row])
    count+=0
    if count==10:
        break

In [ ]:
# salvando alterações
sheet['B10'].value = "TESTANDO"

# salvar a alteração
wb.save(os.path.join(diretorio, arquivo))

O estilo ficou com problema. Vamos arrumar

In [ ]:
from copy import copy, deepcopy

# copia o estilo
celula = sheet['B10']
original = sheet['C10']
celula._style = copy(original._style)

# salvar a alteração
wb.save(os.path.join(diretorio, arquivo))

### Pandas
Esta biblioteca é realmente a melhor que existe se você quiser fazer contas entre excel. EXCELENTE para análise de dados.

In [ ]:
! pip install xlrd

In [ ]:
import pandas as pd

In [ ]:
arquivo = 'alunos_turma.xlsx'
arquivo = os.path.join(diretorio, arquivo)
df = pd.read_excel(arquivo, sheet_name='QtdeAlunos')

In [ ]:
df

In [ ]:
# média de alunos por turma
df['quantidade de alunos'].mean()

In [ ]:
# média de alunos por turma e por produto
df.groupby('produto')['quantidade de alunos'].mean()

In [ ]:
# carregando a aba de professores
df_prof = pd.read_excel(arquivo, sheet_name='Professores')
df_prof

In [ ]:
# cruzando as abas(PROCV ou VLOOKUP) 
tabela_completa = pd.merge(df, df_prof)
tabela_completa

In [ ]:
# buscando somente algumas colunas
tabela_completa[['data', 'Nome Professor']]

In [ ]:
# alterando o valor da quantidade de alunos para a linha 0
tabela_completa.iloc[0,3] = 15
tabela_completa

In [ ]:
# passar linha a linha
for linha in tabela_completa.iterrows():
    print("Id da linha: {}".format(linha[0]))
    print("Id da linha dados:\n{}".format(linha[1]))

In [ ]:
# acessando um campo da linha
dados = linha[1]
dados['data'], dados['produto']

O Pandas é realmente algo fora de série. Neste [post](https://paulovasconcellos.com.br/28-comandos-%C3%BAteis-de-pandas-que-talvez-voc%C3%AA-n%C3%A3o-conhe%C3%A7a-6ab64beefa93) você pode ver algumas dicas de como usar a biblioteca.

Também existem muitos tutoriais na internet. Entre eles:

- [PandasTour](https://nbviewer.jupyter.org/gist/wesm/4757075/PandasTour.ipynb)
- [Análise estatística com Pandas](https://github.com/fonnesbeck/statistical-analysis-python-tutorial)
- [Gráficos Simples](https://pbpython.com/simple-graphing-pandas.html)
- [Automatizando a tarefa repetitiva de MSOffice](https://ep2018.europython.eu/media/conference/slides/the-boring-python-office-talk-automate-powerpoint-excel-and-pdf-with-python.pdf)

### PowerPoint
Podemos também criar e alterar Power Points com o Python. Seguem abaixo alguns exemplos 

In [ ]:
# instala a biblioteca do python para manipular Power Point
!pip install python-pptx

In [ ]:
from pptx import Presentation

# abrindo um ppt que já existe
arquivo = 'ApresentaçãoExemplo.pptx'
arquivo = os.path.join(diretorio, arquivo)
arquivo_saida = os.path.join(diretorio, 'Modelo.pptx')
ppt = Presentation(arquivo)
ppt

In [ ]:
def analizarPPT(input, output):
    """ 
    Fonte: https://pbpython.com/creating-powerpoint.html
    Lê um powerpoint e pega os layouts mestres, gerando um novo ppt que mostrará os nomes e títulos do layout mestre
    """
    prs = Presentation(input)
    # para cada um dos slides
    for index, _ in enumerate(prs.slide_layouts):
        # adiciona na nova apresentação um slide igual ao layout
        layout = prs.slide_layouts[index]
        print(layout.name)
        slide = prs.slides.add_slide(layout)

        # verifica se para o controle (shape) existe um título
        try:
            title = slide.shapes.title
            title.text = 'Title for Layout {}'.format(index)
        except AttributeError:
            print("No Title for Layout {}".format(index))
            
        # para cada um dos controles
        for shape in slide.placeholders:
            # se for um placeholder
            if shape.is_placeholder:
                phf = shape.placeholder_format
                # nao apaga o titulo caso ele exista
                try:
                    if 'Title' not in shape.text:
                        shape.text = 'Placeholder index:{} type:{}'.format(phf.idx, shape.name)
                except AttributeError:
                    print("{} has no text attribute".format(phf.type))
                print('{} {}'.format(phf.idx, shape.name))
    prs.save(output)
analizarPPT(arquivo, arquivo_saida)

In [ ]:
def buscarComentarios(input):
    """
    Busca todos os comentarios da apresentacao
    """
    prs = Presentation(input)
    for i, slide in enumerate(prs.slides):
        # imprime o comentário
        if slide.has_notes_slide:
            comentario = slide.notes_slide.notes_text_frame
            print("Notas do slide {}: {}".format(i, comentario.text))
buscarComentarios(arquivo)

In [ ]:
# adicionando uma figura no slide novo
ppt = Presentation(arquivo)
slide = ppt.slides.add_slide(ppt.slide_layouts[-1])

In [ ]:
# pega o placeholder (o local reservado) para a imagem
for ph in slide.placeholders:
    print(ph)
foto = list(slide.placeholders)[1]

# coloca um titulo
titulo = list(slide.placeholders)[0]
titulo.text = 'Agora este slide ficou lindão'

In [ ]:
%matplotlib inline
nome_figura = os.path.join(diretorio, 'grafico.png')

# cria uma figura para adicionar (vinda do exemplo do Python)
ax = df.groupby('produto')['quantidade de alunos'].mean().plot(kind='bar', title='Numero de alunos por produto')
fig = ax.get_figure()
fig.savefig(nome_figura)

# adiciona no placeholder
foto.insert_picture(nome_figura)
ppt.save(arquivo)

Abaixo algumas referências interessantes:

- [Criando um modelo através de uma apresentação](https://pbpython.com/creating-powerpoint.html)
